In [1]:
import math
import pandas as pd
import numpy as np
import sys
import csv
import statistics
from functools import reduce
from scipy import stats

# Make it so really large files don't cause the system to error out
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

df1 = pd.read_excel("Myc_Fold_Change_and_P_value.xlsx")
df2 = pd.read_excel("Affy_Gene_IDs.xlsx")
df = pd.merge(df1, df2, on=['ID'])
df.dropna(axis='rows', inplace=True)
df = df.reset_index(drop=True)
a = df.filter(like='EMT')
b = df.filter(like='Squamous')
c = df.filter(like='Microacinar')
ab = (df.filter(regex='EMT|Squamous'))
ac = (df.filter(regex='EMT|Microacinar'))
bc = (df.filter(regex='Squamous|Microacinar'))
EMT = stats.ttest_ind(a,bc, axis=1)
np.savetxt("EMT_Test_Statistic.csv", EMT, delimiter=",")
EMT_stats = pd.read_csv("EMT_Test_Statistic.csv", sep=",", header=None).T
df["EMT_T-statistic"] = EMT_stats[0]
df["EMT_P-value"] = EMT_stats[1]
Squamous = stats.ttest_ind(b,ac, axis=1)
np.savetxt("Squamous_Test_Statistic.csv", Squamous, delimiter=",")
Squamous_stats = pd.read_csv("Squamous_Test_Statistic.csv", sep=",", header=None).T
df["Squamous_T-statistic"] = Squamous_stats[0]
df["Squamous_P-value"] = Squamous_stats[1]
Microacinar = stats.ttest_ind(c,ab, axis=1)
np.savetxt("Microacinar_Test_Statistic.csv", Microacinar, delimiter=",")
Microacinar_stats = pd.read_csv("Microacinar_Test_Statistic.csv", sep=",", header=None).T
df["Microacinar_T-statistic"] = Microacinar_stats[0]
df["Microacinar_P-value"] = Microacinar_stats[1]
df["EMT_Mean"] = a.mean(axis=1)
df["Squamous_Mean"] = b.mean(axis=1)
df["Microacinar_Mean"] = c.mean(axis=1)
df["EMT_Squamous_Mean"] = ab.mean(axis=1)
df["EMT_Microacinar_Mean"] = ac.mean(axis=1)
df["Squamous_Microacinar_Mean"] = bc.mean(axis=1)
df["Log2FC_EMT"] = np.log2((df['EMT_Mean'] / df['Squamous_Microacinar_Mean']))
df["Neg_Log10pv_EMT"] = -np.log10(df["EMT_P-value"])
df["Log2FC_Squamous"] = np.log2((df['EMT_Squamous_Mean'] / df['EMT_Microacinar_Mean']))
df["Neg_Log10pv_Squamous"] = -np.log10(df["Squamous_P-value"])
df["Log2FC_Microacinar"] = np.log2((df['Microacinar_Mean'] / df['EMT_Squamous_Mean']))
df["Neg_Log10pv_Microacinar"] = -np.log10(df["Microacinar_P-value"])
df.to_csv("Myc_Gene_Expression_Fold_Changes.csv")
print("done")

C:\Users\Andrechek Lab\Documents\MobaXterm\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


done


In [61]:
import math
import pandas as pd
import numpy as np
import sys
import csv
import statistics
from functools import reduce
from scipy import stats

# Make it so really large files don't cause the system to error out
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

# Mouse data
df1 = pd.read_csv("GSE15904_series_matrix_CB.txt", sep='\t', index_col='ID')
df1 = df1.T
df1["Batch"] = 1
df1 = df1.T
df2 = pd.read_csv("GSE42533_series_matrix_CB.txt", sep='\t', index_col='ID')
df2 = df2.T
df2["Batch"] = 2
df2 = df2.T
df3 = pd.read_csv("GSE104397_series_matrix_CB.txt", sep='\t', index_col='ID')
df3 = df3.T
df3["Batch"] = 3
df3 = df3.T
df12 = pd.merge(df1, df2, left_index=True, right_index=True)
df13 = pd.merge(df12, df3, left_index=True, right_index=True)
df13 = df13.T
batch = df13.pop("Batch")
df13 = df13.T

df_affy = pd.read_excel("Affy_Gene_IDs.xlsx", usecols=["ID", "Gene Symbol"], index_col='ID')
df_affy.dropna(axis='rows', inplace=True)
df = pd.merge(df_affy, df13, on=['ID'])
gene_symbol = df.pop("Gene Symbol")
df.insert(0, "Gene Symbol", gene_symbol)
df = df.T
df.insert(0, "Batch", batch)
df = df.T
df["Gene Symbol"] = df["Gene Symbol"].fillna("Batch")
df = df.rename(columns={"Gene Symbol": "Hugo_Symbol"})
df = df.drop_duplicates(subset=["Hugo_Symbol"], keep='first')
df = df.set_index("Hugo_Symbol")
df = df.T
df["Subtype"] = df.index.str.replace('[^a-zA-Z]', '')
col = df.pop("Subtype")
df = df.astype(dtype="float64")
df.insert(0, "Subtype", col)
df = df.T
df.index = df.index.str.upper()

# Metabric data
df1 = pd.read_csv("data_mrna_agilent_microarray.txt", sep="\t")
df3 = df1.T
df2 = pd.read_csv("data_clinical_patient.txt", sep="\t", skiprows=4, usecols=[0,15])
df2 = df2.set_index('PATIENT_ID')
df4 = pd.merge(df3, df2, left_index=True, right_index=True)
claudin = df4.pop("CLAUDIN_SUBTYPE")
df4 = df4.astype(dtype="float64")
df4 = df4.T
df4 = df4.set_index(df1["Hugo_Symbol"])
df4 = df4.T
df4.insert(0, "Subtype", claudin)
df4["Batch"] = 4
df4 = df4.T
df4.index = df4.index.str.upper()

# Merge Mouse and Human Data
df5 = pd.merge(df, df4, on=['Hugo_Symbol'])
df5.to_csv("Raw_Human_and_Mouse_Combined_Gene_Exp_Data.csv")


C:\Users\ANDREC~1\AppData\Local\Temp/ipykernel_20304/4255273032.py:55: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Subtype"] = df.index.str.replace('[^a-zA-Z]', '')
